This notebook is for analyzing and cleaning the the entire dataset.

## Summary


The following data set if for an ecommerce company that sells tires.  The data set is from Google paid search queries related to Product Listing Ads (PLAs) which include Google Shopping queries. The search terms that include Client’s brand name have been excluded. Given this data, and any other information you want to add, what can you tell us? What information/visualizations can we share with the customer to help them position/re-position etc. How are people finding their products, and what can we do with this information?

Original Query Parameters:

Campaign Type: Google Shopping PLAs
Network: Google Search; Excluding all other Audience Extension Networks
Note: This file is based upon Search Terms (not Keywords), which are the actual queries that users typed in

Conversion Parameters (e.g., how is Conversions and Conversion Value calculated):

Includes the revenue generated from transactions that occur within 60 days of an ad click; multiple transactions within 60 days are counted
Conversion Values are modeled on a “Position-based attribution model”, which means that Google Ads allocates conversion value to ads based on: First Ad Click = 40%, Last Ad Click = 40%, All Middle Clicks = 20% (Linear Split)
As a result, Conversions and Conversion Values are not a perfect measure as we don’t know if the conversion value was due to a search query being the 1st click, last click, or a middle click

Calculations Not Included:
CV Gross Profit = Conversion Value * 20%
CV Gross Profit Less Ad Cost = (Conversion Value * 20% ) – Ad Cost
MCF Revenue = (Conversion Value * 50%)
MCF Gross Profit = ((Conversion Value * 50%) *20%) - https://www.ppchero.com/ultimate-guide-to-googles-multi-channel-funnels/
MCF Gross Profit Less Ad Cost = ((Conversion Value * 50%) * 20%) – Ad Cost

Single Click w/Measures

Columns:

     - Full Words
     - Impressions
     - Clicks (Drop after confirmation)
     - Measures
         - Tire Type (P, LT or none)
         - Tire Width (first 3 numbers in millimeters)
         - Aspect Ratio (2 digit number after "/", height equal to the % of tire's width.) 
         - Construction (If R is present then the layers run radially across the tire
         - Wheel Diameter (Size of wheel the tire is supposed to fit)
         
         - Unknowns
             - Load Index
             - Speed Rating
             - DOT Sumbol
             - Tire Identification Number
             - UTQG
      - Cost, Conversion_Value, Conversions, Conv_value_cost: Brought but not for model
[Source](https://www.goodyearautoservice.com/en-US/tire-basics/tire-size)
     
no clicks?
few clicks?

 - 8 -> go kart? Trailer or dirt bike
 - 32 -> big rims?
 - 82 -> tractor?
 - 
 ECONOMICS OF NICHES
 
 Truck people are already paying more so might as well increase the margin?
 
 Stock Market effect for aspect ration and dwhich sizes people need for which more popular car
  - Flag for if brand is included.

    P_Type
    LT_Type
    No_Type
    Tire_width
    Aspect_ratio
    Construction
    Wheel_diameter

 - Impression: https://www.klipfolio.com/metrics/difference/ad-clicks-vs-ad-impressions#:~:text=Ad%20Clicks%2C%20or%20simply%20Clicks,to%20reach%20an%20online%20property.&text=Ad%20Impressions%20(IMPR)%20is%20a,screen%20within%20the%20publisher's%20network.
 
 - Impression Share: https://support.google.com/google-ads/answer/2497703?hl=en
  
 - Conversion: A conversion in Google Ads is when a user performs some specified action after clicking an ad or viewing a Display Network ad, such as purchasing a product, installing a mobile app, or signing up for an email list. Conversion tracking provides key insights into users' actions after viewing or clicking an ad.

## Imports & Such

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random as ran
%matplotlib inline
import re
import random

In [21]:
# Imports
# Natural Language Processing libraries, initiations and functions
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import re

In [104]:
df = pd.read_csv('./../G_ad_data/G_ad_clean.csv', index_col= False)
df_raw = pd.read_csv('./../G_ad_data/Search_terms_cleaned.csv')#.columns

C:\Users\Peridot of Earth\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [105]:
print(df.shape)
df.head()

(2249703, 14)


,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost,CV_gross_profit,CV_gross_profit_less_ad_cost,MCF_revenue,MCF_gross_profit,MCF_gross_profit_less_ad_cost
0,285 60r20 sale,2.22,0.74,3,3,0.00,0.00,0.00,0.00,0.000,-2.220,0.00,0.000,-2.220
1,325 45r22 nitto 420s,0.22,0.22,1,1,0.00,0.00,0.00,0.00,0.000,-0.220,0.00,0.000,-0.220
2,wild country at,3.78,0.22,39,17,0.07,6.32,0.43,1.67,1.264,-2.516,3.16,0.632,-3.148
3,h w 100xt primex tires 7.50 16,0.17,0.17,1,1,0.00,0.00,0.00,0.00,0.000,-0.170,0.00,0.000,-0.170
4,285 75r16 at tires,21.69,0.39,263,56,0.63,438.10,1.10,20.20,87.620,65.930,219.05,43.810,22.120


In [106]:
df.drop(['CV_gross_profit', 'CV_gross_profit_less_ad_cost', 
         'MCF_revenue', 'MCF_gross_profit', 'MCF_gross_profit_less_ad_cost'], axis = 1, inplace = True)

df.head(10)

,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost
0,285 60r20 sale,2.22,0.74,3,3,0.00,0.00,0.00,0.00
1,325 45r22 nitto 420s,0.22,0.22,1,1,0.00,0.00,0.00,0.00
2,wild country at,3.78,0.22,39,17,0.07,6.32,0.43,1.67
3,h w 100xt primex tires 7.50 16,0.17,0.17,1,1,0.00,0.00,0.00,0.00
4,285 75r16 at tires,21.69,0.39,263,56,0.63,438.10,1.10,20.20
5,295 65r15,65.89,0.21,3098,314,0.50,106.26,0.16,1.61
6,235 65 17 toyo tires open country,0.76,0.76,1,1,0.00,0.00,0.00,0.00
7,285 70 16 tires for sale,4.84,0.37,21,13,0.00,0.00,0.00,0.00
8,35x12 50x16 5 tires,23.76,0.42,211,56,0.00,0.00,0.00,0.00
9,5x by 30 r20 tire,0.52,0.52,1,1,0.00,0.00,0.00,0.00


In [107]:
df.head(10)

,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost
0,285 60r20 sale,2.22,0.74,3,3,0.00,0.00,0.00,0.00
1,325 45r22 nitto 420s,0.22,0.22,1,1,0.00,0.00,0.00,0.00
2,wild country at,3.78,0.22,39,17,0.07,6.32,0.43,1.67
3,h w 100xt primex tires 7.50 16,0.17,0.17,1,1,0.00,0.00,0.00,0.00
4,285 75r16 at tires,21.69,0.39,263,56,0.63,438.10,1.10,20.20
5,295 65r15,65.89,0.21,3098,314,0.50,106.26,0.16,1.61
6,235 65 17 toyo tires open country,0.76,0.76,1,1,0.00,0.00,0.00,0.00
7,285 70 16 tires for sale,4.84,0.37,21,13,0.00,0.00,0.00,0.00
8,35x12 50x16 5 tires,23.76,0.42,211,56,0.00,0.00,0.00,0.00
9,5x by 30 r20 tire,0.52,0.52,1,1,0.00,0.00,0.00,0.00


## Figuring out Conv/Value

In [108]:
df.head(3)

,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost
0,285 60r20 sale,2.22,0.74,3,3,0.00,0.00,0.00,0.00
1,325 45r22 nitto 420s,0.22,0.22,1,1,0.00,0.00,0.00,0.00
2,wild country at,3.78,0.22,39,17,0.07,6.32,0.43,1.67


In [109]:
6.32 / 3.78
df['Check_diff'] = (np.round((df['Conv_value'] / df['Cost'])) - df['Conv_value_cost']).fillna(0)

In [110]:
df[df['Check_diff'] != 0]['Check_diff']

2          0.33
4         -0.20
5          0.39
11        -0.35
15         0.45
           ... 
2248991   -0.36
2249378   -0.48
2249397    0.27
2249537   -0.47
2249632   -0.23
Name: Check_diff, Length: 72184, dtype: float64


total conversions/ total cost of all ad interactions.

It’s calculated by dividing your total conversion value by the total cost of all ad interactions.

Total Cost:   2876546
Total Value: 23596698   

In [111]:
23596698 / 2876546

8.203135983224325

23596698 - 2876546
Conversion Value - Cost
Conv Value/Cost - > Not just Value minus cost.

In [112]:
temp_df = pd.DataFrame(columns= ['Column','Total_of_value','Rows != 0'])
for col in df.drop(['Search_term'], axis = 1).columns:
    temp_df = temp_df.append({
        'Column' : col, 
        'Total_of_value' : np.round(df[col].T.sum()),
        'Rows != 0' : df[df[col] != 0].shape[0]}, ignore_index = True)
temp_df

,Column,Total_of_value,Rows != 0
0,Cost,2876546.0,2203806
1,Avg_cpc,738352.0,2188022
2,Impressions,95671871.0,2204351
3,Clicks,7761976.0,2188022
4,Conversions,60783.0,73997
5,Conv_value,23596698.0,74510
6,Conv_rate,1779750.0,74420
7,Conv_value_cost,64754878.0,74452
8,Check_diff,inf,72184


64 754 878

## Cleaning

In [113]:
def funct_map(x):
    if x > 1:
        x = 2
    return x
df['Clicks_bin'] = df['Clicks'].map(lambda x: funct_map(x))
df['Clicks_bin'].value_counts()

1    1500697
2     687325
0      61681
Name: Clicks_bin, dtype: int64

In [114]:
df.to_csv('./../G_ad_data/Meta_all_2_4.csv', index= False)

In [115]:
#df_temp = 
df.drop(['Search_term', 'Impressions', 'Clicks_bin'], axis = 1)#inplace = True)

,Cost,Avg_cpc,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost,Check_diff
0,2.22,0.74,3,0.00,0.00,0.00,0.00,0.00
1,0.22,0.22,1,0.00,0.00,0.00,0.00,0.00
2,3.78,0.22,17,0.07,6.32,0.43,1.67,0.33
3,0.17,0.17,1,0.00,0.00,0.00,0.00,0.00
4,21.69,0.39,56,0.63,438.10,1.10,20.20,-0.20
...,...,...,...,...,...,...,...,...
2249698,0.68,0.68,1,0.00,0.00,0.00,0.00,0.00
2249699,0.52,0.26,2,0.00,0.00,0.00,0.00,0.00
2249700,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00
2249701,0.35,0.18,2,0.00,0.00,0.00,0.00,0.00


In [116]:
0.07/ 17

0.00411764705882353

Cost			
Avg_cpc			
Clicks			
Conversions		
Conv_value		
Conv_rate		Conversions / Clicks	*
Conv_value_cost	Conv Value  / Cost

.63/56

## Filtering to test

In [118]:
print('None:     ', df[df['Clicks'] == 0].shape[0])
print('Singles:', df[df['Clicks'] == 1].shape[0])
print('2 - 10:  ', df[(df['Clicks'] > 1) & (df['Clicks'] <= 10)].shape[0])
print('>10:      ', df[df['Clicks'] > 10].shape[0])

None:      61681
Singles: 1500697
2 - 10:   612473
>10:       74852


In [119]:
# Counting how many Measurements Numbersets are present
list_lists_count = []
for num in list_results:
    list_lists_count.append(len(num))

for num in range(6):
    print(num, list_lists_count.count(num))

0 1260187
1 982745
2 6750
3 19
4 2
5 0


In [120]:
df['Tire_measures'] = list_results

In [121]:
df['list_counts'] = list_lists_count

In [122]:
df = df[df['list_counts'] == 1]
print(df.shape)
df.head()

(982745, 13)


,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost,Check_diff,Clicks_bin,Tire_measures,list_counts
0,285 60r20 sale,2.22,0.74,3,3,0.00,0.00,0.00,0.00,0.00,2,[285 60r20],1
1,325 45r22 nitto 420s,0.22,0.22,1,1,0.00,0.00,0.00,0.00,0.00,1,[325 45r22],1
4,285 75r16 at tires,21.69,0.39,263,56,0.63,438.10,1.10,20.20,-0.20,2,[285 75r16],1
5,295 65r15,65.89,0.21,3098,314,0.50,106.26,0.16,1.61,0.39,2,[295 65r15],1
6,235 65 17 toyo tires open country,0.76,0.76,1,1,0.00,0.00,0.00,0.00,0.00,1,[235 65 17],1


In [123]:
df['Tire_measures'] = df['Tire_measures'].map(lambda x: x[0])

In [124]:
df.head()

,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost,Check_diff,Clicks_bin,Tire_measures,list_counts
0,285 60r20 sale,2.22,0.74,3,3,0.00,0.00,0.00,0.00,0.00,2,285 60r20,1
1,325 45r22 nitto 420s,0.22,0.22,1,1,0.00,0.00,0.00,0.00,0.00,1,325 45r22,1
4,285 75r16 at tires,21.69,0.39,263,56,0.63,438.10,1.10,20.20,-0.20,2,285 75r16,1
5,295 65r15,65.89,0.21,3098,314,0.50,106.26,0.16,1.61,0.39,2,295 65r15,1
6,235 65 17 toyo tires open country,0.76,0.76,1,1,0.00,0.00,0.00,0.00,0.00,1,235 65 17,1


In [125]:
df['test_checking'] = df['Tire_measures'].map(lambda x: len(x))#.value_counts()

In [126]:
df['test_checking'].value_counts()

9     645361
10    222748
12     49419
11     47351
13      8479
8       6198
14      3167
15        19
16         3
Name: test_checking, dtype: int64

In [130]:
# Instantiating tire width and number of potential tire widths
df['Tire_width'] = df['Tire_measures'].map( lambda x: re.findall(r"\d{3} ", x))
df['NUM_tire_widths'] = df['Tire_width'].map( lambda x: len(x))

In [131]:
# THESE PRESUMED MEASURE VALUES MIGHT BE INCORRECT
df['Aspect_ratio']   = df['Tire_measures'].map( 
    lambda x: int(re.findall(r" \d{2}|r\d{2}", x)[0]))

df['Wheel_diameter'] = df['Tire_measures'].map( 
    lambda x: int(re.findall(r" \d{2}|r\d{2}", x)[-1].strip('r')))


In [132]:
def map_funct(x):
    output = False
    if re.search(r"r", x):
        output = True
    return output
df['Radial_construction'] = df['Tire_measures'].map( lambda x: map_funct(x))

In [133]:
def map_funct(x):
    output = False
    if re.search(r"p", x):
        output = True
    return output
df['Passanger_by_US'] = df['Tire_measures'].map( lambda x: map_funct(x))

In [134]:
def map_funct(x):
    output = False
    if re.search(r"lt", x):
        output = True
    return output
df['Light_truck'] = df['Tire_measures'].map( lambda x: map_funct(x))

In [135]:
df.head()

,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost,Check_diff,...,Tire_measures,list_counts,test_checking,Tire_width,NUM_tire_widths,Aspect_ratio,Wheel_diameter,Radial_construction,Passanger_by_US,Light_truck
0,285 60r20 sale,2.22,0.74,3,3,0.00,0.00,0.00,0.00,0.00,...,285 60r20,1,9,[285 ],1,60,20,True,False,False
1,325 45r22 nitto 420s,0.22,0.22,1,1,0.00,0.00,0.00,0.00,0.00,...,325 45r22,1,9,[325 ],1,45,22,True,False,False
4,285 75r16 at tires,21.69,0.39,263,56,0.63,438.10,1.10,20.20,-0.20,...,285 75r16,1,9,[285 ],1,75,16,True,False,False
5,295 65r15,65.89,0.21,3098,314,0.50,106.26,0.16,1.61,0.39,...,295 65r15,1,9,[295 ],1,65,15,True,False,False
6,235 65 17 toyo tires open country,0.76,0.76,1,1,0.00,0.00,0.00,0.00,0.00,...,235 65 17,1,9,[235 ],1,65,17,False,False,False


## Measure Column Cleaning

In [136]:
df = df[df['NUM_tire_widths'] == 1]

In [137]:
df['Tire_width'] = df['Tire_width'].map( lambda x: int(x[0].strip(' ')))

<ipython-input-137-dc8d4acb768b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tire_width'] = df['Tire_width'].map( lambda x: int(x[0].strip(' ')))


In [138]:
df.head()

,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost,Check_diff,...,Tire_measures,list_counts,test_checking,Tire_width,NUM_tire_widths,Aspect_ratio,Wheel_diameter,Radial_construction,Passanger_by_US,Light_truck
0,285 60r20 sale,2.22,0.74,3,3,0.00,0.00,0.00,0.00,0.00,...,285 60r20,1,9,285,1,60,20,True,False,False
1,325 45r22 nitto 420s,0.22,0.22,1,1,0.00,0.00,0.00,0.00,0.00,...,325 45r22,1,9,325,1,45,22,True,False,False
4,285 75r16 at tires,21.69,0.39,263,56,0.63,438.10,1.10,20.20,-0.20,...,285 75r16,1,9,285,1,75,16,True,False,False
5,295 65r15,65.89,0.21,3098,314,0.50,106.26,0.16,1.61,0.39,...,295 65r15,1,9,295,1,65,15,True,False,False
6,235 65 17 toyo tires open country,0.76,0.76,1,1,0.00,0.00,0.00,0.00,0.00,...,235 65 17,1,9,235,1,65,17,False,False,False


In [141]:
df.drop(['test_checking', 'NUM_tire_widths','list_counts'], axis = 1, inplace = True)

C:\Users\Peridot of Earth\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [142]:
df.to_csv('./../G_ad_data/Meta_sans_nonmeasures_2_3.csv', index= False)

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.drop(['Conversions', 'Conv_rate','list_counts', 'test_checking', 'NUM_tire_widths', 'Tire_measures'], 
        axis = 1, inplace = True)

In [ ]:
print(df.shape)
df.head()

In [ ]:
# Preprocessing
def preprocess(text):
    text = re.sub(r'[^a-z]',' ', text)
    tokens = word_tokenize(text)
    lemmer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    return " ".join([lemmer.lemmatize(word) for word 
                     in tokens if len(word) > 5 and not word in stop_words])



# Cvec, Standard
cvec = CountVectorizer(analyzer = "word",
                       min_df = 1000,
                       preprocessor = preprocess,
                       stop_words = 'english') 
# Cvec DF
df_words = pd.DataFrame(cvec.fit_transform(df['Search_term']).todense(), 
                        columns=cvec.get_feature_names())

In [ ]:
df_words['bfgoodrich'].sum()

In [ ]:
df_words.sum().sort_values(ascending=False).head(20)

In [ ]:
#df_words.to_csv('./../G_ad_data/Current_WORDS_analysis_1_31.csv', index= False)

In [ ]:
df.head()

In [ ]:
# SNS
plt.figure(figsize= (9,9))
sns.heatmap(df.corr(), 
            #cbar=False, 
            linewidths= .01, 
            linecolor='black', 
            cmap=sns.color_palette("Purples"),
            annot=True);

In [ ]:
np.correlate(df[''])

    - Measures
         - Tire Type (P, LT or none)
         - Tire Width (first 3 numbers in millimeters)
         - Aspect Ratio (2 digit number after "/", height equal to the % of tire's width.) 
         - Construction (If R is present then the layers run radially across the tire
         - Wheel Diameter (Size of wheel the tire is supposed to fit)

p?l?t?\d{3} \d{2}.?\d{0,3} ?r? ?\d{2}
New Rows
    P_Type
    LT_Type
    No_Type
    Tire_width
    Aspect_ratio
    Construction
    Wheel_diameter

## Where's all the money gone?
Lets account for all [] of the cost and value

## Where there's no clicks

In [ ]:
df_clickless = df[df['Clicks'] == 0]
print(df_clickless.shape)
df_clickless.head()


In [143]:
df_clickless.shape

(61681, 11)

In [145]:
temp_df_clickless = pd.DataFrame(columns= ['Column','Total_of_value','Rows != 0'])
for col in df_clickless.drop(['Search_term'], axis = 1).columns:
    temp_df_clickless = temp_df_clickless.append({
        'Column' : col, 
        'Total_of_value' : np.round(df_clickless[col].T.sum()),
        'Rows != 0' : df_clickless[df_clickless[col] != 0].shape[0]}, ignore_index = True)
temp_df_clickless

,Column,Total_of_value,Rows != 0
0,Cost,2605.0,15784
1,Avg_cpc,0.0,0
2,Impressions,65184.0,21743
3,Clicks,0.0,0
4,Conversions,93.0,151
5,Conv_value,34242.0,151
6,Conv_rate,6692.0,102
7,Conv_value_cost,236379.0,101
8,Check_diff,inf,141
9,Clicks_bin,0.0,61681


In [34]:
df_clickless[df_clickless['Conv_rate'] != 0]["Conv_rate"].value_counts()

100.00    29
40.00     25
50.00     16
200.00     7
5.00       4
30.00      4
80.00      4
6.67       3
10.00      3
0.74       1
3.33       1
1.18       1
21.67      1
1.54       1
33.33      1
20.00      1
Name: Conv_rate, dtype: int64

In [35]:
df_clickless[df_clickless['Conv_rate'] != 0]["Conv_rate"].value_counts()

100.00    29
40.00     25
50.00     16
200.00     7
5.00       4
30.00      4
80.00      4
6.67       3
10.00      3
0.74       1
3.33       1
1.18       1
21.67      1
1.54       1
33.33      1
20.00      1
Name: Conv_rate, dtype: int64

In [150]:
temp_df_clickless.drop([1,3,9])

,Column,Total_of_value,Rows != 0
0,Cost,2605.0,15784
2,Impressions,65184.0,21743
4,Conversions,93.0,151
5,Conv_value,34242.0,151
6,Conv_rate,6692.0,102
7,Conv_value_cost,236379.0,101
8,Check_diff,inf,141


## Multiclicks

In [79]:
temp_df = pd.DataFrame(columns= ['Column','Total_of_value','Rows != 0'])
for col in df.drop(['Search_term'], axis = 1).columns:
    temp_df = temp_df.append({
        'Column' : col, 
        'Total_of_value' : np.round(df[col].T.sum()),
        'Rows != 0' : df[df[col] != 0].shape[0]}, ignore_index = True)
temp_df

,Column,Total_of_value,Rows != 0
0,Cost,2876546.0,2203806
1,Avg_cpc,738352.0,2188022
2,Impressions,95671871.0,2204351
3,Clicks,7761976.0,2188022
4,Conversions,60783.0,73997
5,Conv_value,23596698.0,74510
6,Conv_rate,1779750.0,74420
7,Conv_value_cost,64754878.0,74452
8,Check_diff,inf,72184
9,Clicks_bin,2875347.0,2188022


In [80]:
temp_df = pd.DataFrame(columns= ['Column','Total_of_value','Rows != 0'])
for col in df.drop(['Search_term'], axis = 1).columns:
    temp_df = temp_df.append({
        'Column' : col, 
        'Total_of_value' : np.round(df[col].T.sum()),
        'Rows != 0' : df[df[col] != 0].shape[0]}, ignore_index = True)
temp_df

,Column,Total_of_value,Rows != 0
0,Cost,2876546.0,2203806
1,Avg_cpc,738352.0,2188022
2,Impressions,95671871.0,2204351
3,Clicks,7761976.0,2188022
4,Conversions,60783.0,73997
5,Conv_value,23596698.0,74510
6,Conv_rate,1779750.0,74420
7,Conv_value_cost,64754878.0,74452
8,Check_diff,inf,72184
9,Clicks_bin,2875347.0,2188022


In [81]:
# Isolating all potnetial Tire Measurement Numbersets
list_results = []
for row in df['Search_term']:
    result = re.findall(r'p?l?t?\d{3} \d{2}.?\d{0,3} ?r? ?\d{2}', row)#(r'[\d.r \/]+') #(r'[\d.rbyhpx ]+')
    
    list_inner = []
    for item in result:
        item = item.strip(' ')
        #if len(item) > 7 & len(item) < 12:
        list_inner.append(item)
    list_results.append(list_inner)

In [144]:
temp_df

,Column,Total_of_value,Rows != 0
0,Cost,2876546.0,2203806
1,Avg_cpc,738352.0,2188022
2,Impressions,95671871.0,2204351
3,Clicks,7761976.0,2188022
4,Conversions,60783.0,73997
5,Conv_value,23596698.0,74510
6,Conv_rate,1779750.0,74420
7,Conv_value_cost,64754878.0,74452
8,Check_diff,inf,72184
